## Building A Chatbot
Today, We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This notebook will cover the basics which will be helpful for those two more advanced topics.

In [9]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [10]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024792D64740>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002479D0749B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.messages import HumanMessage, AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is iqra and i am AI engineer"),
        AIMessage(content="Hello iqra! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey, whats my name and what do i do")
    ]
)

AIMessage(content="You are iqra, and you are an AI Engineer! \n\nWe met just a few moments ago, and you told me about your work.  😊  \n\nIs there anything else you'd like to tell me about your role as an AI Engineer?  I'm curious to hear more!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 95, 'total_tokens': 160, 'completion_time': 0.118181818, 'prompt_time': 0.004323235, 'queue_time': 0.019422725000000002, 'total_time': 0.122505053}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2fc7ae00-b62e-45ca-b335-28d9fe0fd259-0', usage_metadata={'input_tokens': 95, 'output_tokens': 65, 'total_tokens': 160})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [15]:
config={"configurable":{"session_id":"chat1"}}

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is iqra and I am a Chief AI Engineer")],
    config=config
)

In [17]:
response.content

"Hi Iqra, it's nice to meet you! \n\nThat's a very impressive title. As a large language model, I'm always interested in learning more about how people are using AI in innovative ways. \n\nWhat kind of work are you doing as a Chief AI Engineer?  What are some of the most exciting projects you're involved in?\n"

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Iqra.  You told me at the beginning of our conversation! 😊 \n\nIs there anything else I can help you with?  I'm ready for your next question.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 116, 'total_tokens': 159, 'completion_time': 0.078181818, 'prompt_time': 0.005490823, 'queue_time': 0.019330005, 'total_time': 0.083672641}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b5e62732-8470-4e0b-b8c1-1bc2bd3545f1-0', usage_metadata={'input_tokens': 116, 'output_tokens': 43, 'total_tokens': 159})

In [23]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John! It's nice to meet you. 👋  \n\nIs there anything I can help you with today?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Iqra")]})

AIMessage(content="Hello Iqra, it's nice to meet you!  \n\nI'm happy to help with any questions you have. \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 32, 'total_tokens': 74, 'completion_time': 0.076363636, 'prompt_time': 0.002365056, 'queue_time': 0.017420624, 'total_time': 0.078728692}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--48232773-7ec9-4265-ae6d-b0d2ee35ffc5-0', usage_metadata={'input_tokens': 32, 'output_tokens': 42, 'total_tokens': 74})

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [28]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is iqra")],
    config=config
)

response

AIMessage(content="Hi Iqra, it's nice to meet you! \n\nI'm happy to help with any questions you have.  Just ask away! 😊  What can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.002667436, 'queue_time': 0.053334814, 'total_time': 0.0863038}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec18f2cd-008c-4847-8fea-171eeebf802a-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [29]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Iqra.  I remember!  😊  \n\nIs there anything else I can help you with?\n'

In [30]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [31]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Iqra")],"language":"Urdu"})
response.content

'اسلام علیکم Iqra! \n\nخوش آمدید! میں آپ کی مدد کرنے کے لیے تیار ہوں۔ \n\nکیا آپ مجھ سے کوئی سوال پوچھنا چاہتے ہیں؟ 😊 \n\n'

In [32]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am iqra")],"language":"urdu"},
    config=config
)
repsonse.content 

'سلام اِقرا! \n\nخوش آمدید. 😊\n\nمیں آپ کی مدد کرنے کے لیے تیار ہوں. کیا آپ سے کوئی سوال ہے؟  \n'

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Urdu"},
    config=config,
)
repsonse.content 

'سلام اِقرا! \n\nخوش آمدید. 😊\n\nمیں آپ کی مدد کرنے کے لیے تیار ہوں. کیا آپ سے کوئی سوال ہے؟  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [36]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [38]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant but not a mind reader, I don't know your favorite ice cream flavor! What's your favorite? 😊  \n"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".  \n\n\n\nLet me know if you\'d like to try another one! 😊\n'

In [46]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [50]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have memory of past conversations or personal information about you. So I don't know your name. 😊\n\nWould you like to tell me?\n"

In [51]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"